# iGD: an intergrated genomic data source


In [221]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob, functools, tqdm, PIL
import time
from multiprocess import Pool
#chr1:  248,956,422+12,151,146-->15,937*16384
#chr2:  242,193,529+12,945,965-->15,573
#chr3:  198,295,559+10,638,715-->12,753
#chr4:  190,214,555+10,165,685-->12,231
#chr5:  181,538,259+ 9,519,995-->11,662
#chr6:  170,805,979+ 9,130,476-->10,983
#chr7:  159,345,973+ 8,613,298-->10,252
#chr8:  145,138,636+ 8,221,520--> 9,361
#chr9:  138,394,717+ 6,590,811--> 8,850
#chr10: 133,797,422+ 7,223,944--> 8,608
#chr11: 135,086,622+ 7,535,370--> 8,705
#chr12: 133,275,309+ 7,228,129--> 8,576
#chr13: 114,364,328+ 5,082,574--> 7,291
#chr14: 107,043,718+ 4,865,950--> 6,831
#chr15: 101,991,189+ 4,515,076--> 6,501
#chr16:  90,338,345+ 5,101,702--> 5,826
#chr17:  83,257,441+ 4,614,972--> 5,364
#chr18:  80,373,285+ 4,035,966--> 5,152
#chr19:  58,617,616+ 3,858,269--> 3,814
#chr20:  64,444,167+ 3,439,621--> 4,144
#chr21:  46,709,983+ 2,049,697--> 2,977
#chr22:  50,818,468+ 2,135,311--> 3,233
#chrX:  156,040,895+ 5,753,881--> 9,876
#chrY:   57,227,415+   211,643--> 3,506
#0. Prepare:
# file name base: blocksize 2**14=16384 bps
fileBase = "bb14"        #14 bits block
nmax = [15937, 15573,12753, 12231, 11662, 10983, 10252, 9361, 8850, 8608, 8705, 
        8576, 7291, 6831, 6501, 5826, 5364, 5152, 3814, 4144, 2977, 3233, 9876, 3506]
folder = ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10', 'chr11', 'chr12', 
          'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22', 'chrX', 'chrY']
#[0, 15937, 24545, 33250, 41826, 49117, 55948, 62449, 68275, 73639, 78791, 82605, 98178, 102322, 105299, 
#108532, 121285, 133516, 145178, 156161, 166413, 175774, 184624, 194500, 198006]

In [222]:
#Append genomic objects to igd file
#filename=filebase+chr_+n.igd (n*16384, n>=0) 
#i: index of the genomic object (eg, ChIP_seq data)
def append_index(filebase, igDf):
    #tmpInfo is a pandas data frame
    igDf.to_csv('igdata/igd_index.tsv', mode='a', sep='\t', header=False)
    return

def append_igd(filebase, tmpData):
    #open files and apend region data
    for ichr in range(0, 24):
        if ichr==22:
            cchr = 'chrX'
        elif ichr==23:
            cchr = 'chrY'
        else:
            cchr = 'chr'+str(ichr+1)
        for m in range(nmax[ichr], nmax[ichr+1]):
            file = open('igdata/'+cchr+'/'+fileBase+'_'+str(m-nmax[ichr])+'.igd', 'a')
            file.append(tmpData[m])
            file.close()
    return
    
def append_igc(data):
    
    return

In [223]:
#Create encode_tfbs .igd
def create_igd():
    #1. Read head info
    file_path = "/home/john/LOLA/LOLACore/hg19/encode_tfbs/"
    file = open(file_path+"index.txt")
    headInfo = pd.read_csv(file, delimiter='\t')
    file.close()
    headInfo.to_csv('igdata/igd_index.tsv', sep='\t')
    
    #2. Read region data
    file_path += "regions/"
    file_ids = next(os.walk(file_path))[2]
    file_ids.sort()
    n_files = len(file_ids)
        
    tmpd = []
    for m in range(0,24):
        tmpd.append(np.empty(nmax[m], dtype=object))
    for i, id_ in tqdm.tqdm(enumerate(file_ids)):
        file = file_path + id_
        regionData = pd.read_csv(file, delimiter='\t', header=None)       
        df = regionData.sort_values(by=[0, 1])   #first by str, then by start
        n1 = df[1].values//16384
        n2 = df[2].values//16384-n1   
        #idx2= np.where(n2>0)[0]
        rchr, ridx, rcnt = np.unique(df[0].values, return_index=True, return_counts=True)        
        #if a record crosses the block boundary, list it under both blocks (duplicates)
        #the start and end values are kept for fast processing (np): serialization and deserial..
        rc1 = [str(x)+'_' for x in df[1].values]
        rc2 = [str(x)+'_' for x in df[2].values]
        rc3 = [str(x)+':' for x in df[4].values]
        #recs =rc1+rc2+rc3

        if i==0:
            for m in range(0, len(rchr)):
                if rchr[m] == 'chrX':
                    ichr = 22
                elif rchr[m] == 'chrY':
                    ichr = 23
                else:
                    ichr = int(rchr[m][3:])-1
                for k in range(0, rcnt[m]):
                    idx0 = k+ridx[m]
                    idx = n1[idx0]
                    rec = rc1[idx0]+rc2[idx0]+rc3[idx0]#recs[idx0] #
                    if n2[idx0] == 0:
                        if tmpd[ichr][idx]==None:
                            tmpd[ichr][idx] = rec
                        else:
                            tmpd[ichr][idx] += rec
                    else:
                        for j in range(0,n2[idx0]):
                            if tmpd[ichr][idx+j]==None:
                                tmpd[ichr][idx+j] = rec
                            else:
                                tmpd[ichr][idx+j] += rec            
            for m in range(0, 24):
                for k in range(0, nmax[m]):
                    if tmpd[m][k]==None:
                        tmpd[m][k] = '|'
                    else:
                        tmpd[m][k] += '|'   
        else:        
            for m in range(0, len(rchr)):
                if rchr[m] == 'chrX':
                    ichr = 22
                elif rchr[m] == 'chrY':
                    ichr = 23
                else:
                    ichr = int(rchr[m][3:])-1
                for k in range(0, rcnt[m]):
                    idx0 = k+ridx[m]
                    idx = n1[idx0]
                    rec = rc1[idx0]+rc2[idx0]+rc3[idx0] #recs[idx0] #
                    if n2[idx0] == 0:
                        tmpd[ichr][idx] += rec
                    else:
                        for j in range(0,n2[idx0]):
                            tmpd[ichr][idx+j] += rec   
            for m in range(0, 24):            
                tmpd[m] += '|'

    #save all files
    for m in range(0, 24):
        for k in range(0, nmax[m]):
            file = open('igdata/'+folder[m]+'/'+fileBase+'_'+str(k)+'.igd', 'w')
            file.write(tmpd[m][k])
            file.close()
    

In [ ]:
#Create encode_tfbs binary data .igb
def create_igb():
    import struct
    
    
    #1. Read head info
    file_path = "/home/john/LOLA/LOLACore/hg19/encode_tfbs/"
    file = open(file_path+"index.txt")
    headInfo = pd.read_csv(file, delimiter='\t')
    file.close()
    headInfo.to_csv('igdata/igd_index.tsv', sep='\t')
    
    #2. Read region data
    file_path += "regions/"
    file_ids = next(os.walk(file_path))[2]
    file_ids.sort()
    n_files = len(file_ids)
        
    tmpd = []
    for m in range(0,24):
        tmpd.append(np.empty(nmax[m], dtype=object))
    for i, id_ in tqdm.tqdm(enumerate(file_ids)):
        file = file_path + id_
        regionData = pd.read_csv(file, delimiter='\t', header=None)       
        df = regionData.sort_values(by=[0, 1])   #first by str, then by start
        n1 = df[1].values//16384
        n2 = df[2].values//16384-n1   
        #idx2= np.where(n2>0)[0]
        rchr, ridx, rcnt = np.unique(df[0].values, return_index=True, return_counts=True)        
        #if a record crosses the block boundary, list it under both blocks (duplicates)
        #the start and end values are kept for fast processing (np): serialization and deserial..
        rc1 = [str(x)+'_' for x in df[1].values]
        rc2 = [str(x)+'_' for x in df[2].values]
        rc3 = [str(x)+':' for x in df[4].values]
        #recs =rc1+rc2+rc3

        if i==0:
            for m in range(0, len(rchr)):
                if rchr[m] == 'chrX':
                    ichr = 22
                elif rchr[m] == 'chrY':
                    ichr = 23
                else:
                    ichr = int(rchr[m][3:])-1
                for k in range(0, rcnt[m]):
                    idx0 = k+ridx[m]
                    idx = n1[idx0]
                    rec = rc1[idx0]+rc2[idx0]+rc3[idx0]#recs[idx0] #
                    if n2[idx0] == 0:
                        if tmpd[ichr][idx]==None:
                            tmpd[ichr][idx] = rec
                        else:
                            tmpd[ichr][idx] += rec
                    else:
                        for j in range(0,n2[idx0]):
                            if tmpd[ichr][idx+j]==None:
                                tmpd[ichr][idx+j] = rec
                            else:
                                tmpd[ichr][idx+j] += rec            
            for m in range(0, 24):
                for k in range(0, nmax[m]):
                    if tmpd[m][k]==None:
                        tmpd[m][k] = '|'
                    else:
                        tmpd[m][k] += '|'   
        else:        
            for m in range(0, len(rchr)):
                if rchr[m] == 'chrX':
                    ichr = 22
                elif rchr[m] == 'chrY':
                    ichr = 23
                else:
                    ichr = int(rchr[m][3:])-1
                for k in range(0, rcnt[m]):
                    idx0 = k+ridx[m]
                    idx = n1[idx0]
                    rec = rc1[idx0]+rc2[idx0]+rc3[idx0] #recs[idx0] #
                    if n2[idx0] == 0:
                        tmpd[ichr][idx] += rec
                    else:
                        for j in range(0,n2[idx0]):
                            tmpd[ichr][idx+j] += rec   
            for m in range(0, 24):            
                tmpd[m] += '|'

    #save all files
    for m in range(0, 24):
        for k in range(0, nmax[m]):
            file = open('igdata/'+folder[m]+'/'+fileBase+'_'+str(k)+'.igd', 'w')
            file.write(tmpd[m][k])
            file.close()
    

In [224]:
#Add files to igd
def add_GObjs(file_path):
    file_ids = next(os.walk(file_path))[2]
    file_ids.sort()
    n_files = len(file_ids)

    tmpd = []
    for m in range(0,24):
        tmpd.append(np.empty(nmax[m], dtype=object))
    for i, id_ in tqdm.tqdm(enumerate(file_ids)):
        file = file_path + id_
        regionData = pd.read_csv(file, delimiter='\t', header=None)       
        df = regionData.sort_values(by=[0, 1])   #first by str, then by start
        n1 = df[1].values//16384
        n2 = df[2].values//16384-n1   
        #idx2= np.where(n2>0)[0]
        rchr, ridx, rcnt = np.unique(df[0].values, return_index=True, return_counts=True)        
        #if a record crosses the block boundary, list it under both blocks (duplicates)
        #the start and end values are kept for fast processing (np)
        rc1 = [str(x)+'_' for x in df[1].values]
        rc2 = [str(x)+'_' for x in df[2].values]
        rc3 = [str(x)+':' for x in df[4].values]
        #recs =rc1+rc2+rc3

        if i==0:
            for m in range(0, len(rchr)):
                if rchr[m] == 'chrX':
                    ichr = 22
                elif rchr[m] == 'chrY':
                    ichr = 23
                else:
                    ichr = int(rchr[m][3:])-1
                for k in range(0, rcnt[m]):
                    idx0 = k+ridx[m]
                    idx = n1[idx0]
                    rec = rc1[idx0]+rc2[idx0]+rc3[idx0]
                    if n2[idx0] == 0:
                        if tmpd[ichr][idx]==None:
                            tmpd[ichr][idx] = rec
                        else:
                            tmpd[ichr][idx] += rec
                    else:
                        for j in range(0,n2[idx0]):
                            if tmpd[ichr][idx+j]==None:
                                tmpd[ichr][idx+j] = rec
                            else:
                                tmpd[ichr][idx+j] += rec            
            for m in range(0, 24):
                for k in range(0, nmax[m]):
                    if tmpd[m][k]==None:
                        tmpd[m][k] = '|'
                    else:
                        tmpd[m][k] += '|'   
        else:        
            for m in range(0, len(rchr)):
                if rchr[m] == 'chrX':
                    ichr = 22
                elif rchr[m] == 'chrY':
                    ichr = 23
                else:
                    ichr = int(rchr[m][3:])-1
                for k in range(0, rcnt[m]):
                    idx0 = k+ridx[m]
                    idx = n1[idx0]
                    rec = rc1[idx0]+rc2[idx0]+rc3[idx0] #recs[idx0] #
                    if n2[idx0] == 0:
                        tmpd[ichr][idx] += rec
                    else:
                        for j in range(0,n2[idx0]):
                            tmpd[ichr][idx+j] += rec   
            for m in range(0, 24):            
                tmpd[m] += '|'

    #save all files
    append_igd(fileBase, tmpd)

In [225]:
#find the overlap of query region set with targets in igd: binary projection
#query is region set, targets--the index of columns of igd
def get_overlap(query, targets):
    #1. determine the files to be loaded from igdc
    
    #2. load the files
    
    #3. project
    
    #4. evaluate
    
    
    return

In [227]:
def zip_igd():
    # test zip file:
    #to zip: zip -r igd.zip chr*
    t0 = time.time()
    import io, zipfile
    archive = zipfile.ZipFile('igdata/igd.zip', 'r')
    #print(archive.infolist()[1:100])
    #print(archive.namelist()[100000:100050]) #infolist()
    #time the retrieval process:
    for m in range(0,24):
        cchr = folder[m]
        for k in range(0, nmax[m]):
            file = cchr+'/bb14_'+str(k)+'.igd'
            data1 = archive.read(file)
    archive.close()
    #print(len(data1), data1[0:5])
    dt = time.time()-t0
    print('dt1=', dt)
    
    #test original
    t0 = time.time()
    file_path = "/home/john/LOLA/LOLACore/hg19/encode_tfbs/regions/"
    file_ids = next(os.walk(file_path))[2]
    for i, id_ in tqdm.tqdm(enumerate(file_ids)):
        regionData = pd.read_csv(file_path+id_, delimiter='\t', header=None)        
        #file = open(file_path + id_, 'r')
        #data1 = file.read() #readlines()
    dt = time.time()-t0
    print('dt2=', dt)   

In [290]:
#0. create igd
#create_igd()
#1. test direct access:
ncols = 689
tmpd = []
for m in range(0,24):
    tmpd.append(np.empty(nmax[m], dtype=object))

t0 = time.time()
for m in range(0,24):
    cchr = folder[m]
    for k in range(0, nmax[m]):
        file = open('igdata/'+cchr+'/bb14_'+str(k)+'.igd', 'r')
        tmpd[m][k] = file.read()              #file.readlines()
dt0 = time.time()-t0

t0 = time.time()
mlist = []
for m in range(0,24):
    rlist=[]
    for k in range(0, nmax[m]):               #split returns a list
        tmp = tmpd[m][k].split('|')
        tmp.pop()                             #pop() return the removed item or del tmp[-1]
        rlist.append(tmp)   #rlist: nmax[m] * ncols : [['1_2_3:2_3_4:', '', '', '7_9_5:', ...], ['5_6_4:',,,], [,,,]]    
    tlist = [''.join(i).strip() for i in zip(*rlist)] #collapse nmax[m] list together-->tlist: 1 * ncols #     [['1_2_3:2_3_4:5_6_4:', '', '', '7_9_5', ...], ['2_3_4',,,], [,,,]]   
    rlist = [x.split(':') for x in tlist]     #rList=ncols lists: [['1_2_3', '2_3_4', ...], ['4_5_6', ...], ....['1_3_5', ...]]
    tlist=[]
    for lst in rlist:
        lst.pop()
        tlist.append([x.split('_') for x in lst])
    mlist.append(tlist)  
dt1 = time.time()-t0

t0 = time.time()
for m in range(24):
    for k in range(ncols):
        mlist[m][k] = np.array(mlist[m][k]).astype(int)
#mlist: 24*689:each is a list of regions   
        
dt2 = time.time()-t0

print(dt0, dt1, dt2)

5.199772596359253 19.697641849517822 11.416809797286987


In [288]:
mlist[0][1]

array([[   858817,    859260,       638],
       [   859954,    860416,      1000],
       [   860309,    860660,       446],
       [   860953,    861476,       592],
       [   861395,    861646,       328],
       [   875698,    876389,      1000],
       [   875698,    876407,       271],
       [   876995,    877399,       742],
       [   911369,    911899,       261],
       [   934695,    935225,       368],
       [   940153,    940683,       396],
       [   975787,    976317,       371],
       [  2160380,   2160910,       336],
       [  3527519,   3528049,       466],
       [  6557534,   6558064,       253],
       [  8483035,   8483565,       335],
       [  8762037,   8762567,       523],
       [  8763358,   8763875,       912],
       [  8763629,   8764159,       250],
       [  9262518,   9262870,       749],
       [  9262835,   9263021,       451],
       [ 10853156,  10853686,       291],
       [ 10855301,  10855433,       465],
       [ 10855362,  10855892,     

In [291]:
def test_read():
    #1. test direct access:
    t0 = time.time()
    total = 0
    for m in range(0,24):
        cchr = folder[m]
        for k in range(0, nmax[m]):
            file = open('igdata/'+cchr+'/bb14_'+str(k)+'.igd', 'rb')
            data1 = file.read() #file.readlines()
            total += len(data1)
    print(len(data1), data1[0:100])
    dt = time.time()-t0
    print('dt2=', dt, ", total=", total)

    #2. test original data:
    t0 = time.time()
    file_path = "/home/john/LOLA/LOLACore/hg19/encode_tfbs/regions/"
    file_ids = next(os.walk(file_path))[2]
    total = 0
    for i, id_ in tqdm.tqdm(enumerate(file_ids)):
        file = open(file_path + id_, 'r')
        data1 = file.read() #readlines()
        total += len(data1)
    print(len(data1), data1[0:100])
    dt = time.time()-t0
    print('dt3=', dt, ", total=", total)